In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import networkx as nx
import itertools
import math as m
from collections import defaultdict

In [2]:
from ranking_funcs import rec_row_prob_back,backdoor_adjustment_opt,find_backdoor_sets_opt,Greedy_Algo,get_most_probable_elements,get_cgm,get_prob_backdoor_opt,get_probs,get_prob_backdoor,filter_prob_df_grouped,read_data,get_ranking_query_prob_grouped,get_stable_ranking_opt,get_ranking_query,get_ranking_query_prob,get_test_revert_ranking_rec,get_new_G,filter_prob_df,calc_prob,base_line

In [3]:
name_df=pd.read_csv('ranking_data_sets/name.basics.tsv', sep='\t')
title_df=pd.read_csv('ranking_data_sets/title.ratings.tsv', sep='\t')

In [8]:
len(name_df['primaryName'].unique())

10129329

In [10]:
len(name_df)

13115801

In [11]:
len(title_df)

1380533

In [12]:
name_df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0072308,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0037382,tt0038355,tt0075213"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0057345,tt0056404,tt0054452,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0078723,tt0072562,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0083922,tt0069467,tt0050986,tt0050976"


In [5]:
title_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2008
1,tt0000002,5.7,270
2,tt0000003,6.5,1925
3,tt0000004,5.4,178
4,tt0000005,6.2,2698


In [13]:

known_titles = name_df['knownForTitles'].str.split(',', expand=True).stack().reset_index(level=1, drop=True)
known_titles.name = 'knownForTitle'

expanded_name_df = name_df.drop(columns=['knownForTitles']).join(known_titles)


merged_df = pd.merge(expanded_name_df, title_df, left_on='knownForTitle', right_on='tconst')


print(merged_df.head())


      nconst     primaryName birthYear deathYear  \
0  nm0000001    Fred Astaire      1899      1987   
1  nm0002045   Stanley Donen      1924      2019   
2  nm0006037  Adolph Deutsch      1897      1980   
3  nm0041509  Michel Auclair      1922      1988   
4  nm0042857  Richard Avedon      1923      2004   

                          primaryProfession knownForTitle     tconst  \
0            soundtrack,actor,miscellaneous     tt0050419  tt0050419   
1           director,miscellaneous,producer     tt0050419  tt0050419   
2      composer,music_department,soundtrack     tt0050419  tt0050419   
3                       actor,miscellaneous     tt0050419  tt0050419   
4  miscellaneous,camera_department,director     tt0050419  tt0050419   

   averageRating  numVotes  
0            7.0     32364  
1            7.0     32364  
2            7.0     32364  
3            7.0     32364  
4            7.0     32364  


In [14]:
merged_df

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitle,tconst,averageRating,numVotes
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous",tt0050419,tt0050419,7.0,32364
1,nm0002045,Stanley Donen,1924,2019,"director,miscellaneous,producer",tt0050419,tt0050419,7.0,32364
2,nm0006037,Adolph Deutsch,1897,1980,"composer,music_department,soundtrack",tt0050419,tt0050419,7.0,32364
3,nm0041509,Michel Auclair,1922,1988,"actor,miscellaneous",tt0050419,tt0050419,7.0,32364
4,nm0042857,Richard Avedon,1923,2004,"miscellaneous,camera_department,director",tt0050419,tt0050419,7.0,32364
...,...,...,...,...,...,...,...,...,...
13695812,nm9993149,June Del Mar,\N,\N,camera_department,tt0137768,tt0137768,7.6,108
13695813,nm9993259,Peter Aufreiter,\N,\N,NaN,tt8724014,tt8724014,7.4,8
13695814,nm9993260,Anna Lisa Genovese,\N,\N,NaN,tt8724014,tt8724014,7.4,8
13695815,nm9993261,Tom Henry,\N,\N,NaN,tt8724014,tt8724014,7.4,8


In [15]:
import pandas as pd

aggregated_df = merged_df.groupby(['tconst', 'averageRating', 'numVotes'])['primaryName'].apply(list).reset_index()


aggregated_df.rename(columns={'primaryName': 'credited_actors'}, inplace=True)


print(aggregated_df)


           tconst  averageRating  numVotes  \
0       tt0000001            5.7      2008   
1       tt0000003            6.5      1925   
2       tt0000004            5.4       178   
3       tt0000005            6.2      2698   
4       tt0000007            5.4       842   
...           ...            ...       ...   
620970  tt9916428            3.5        17   
620971  tt9916460            9.4        18   
620972  tt9916538            8.6         7   
620973  tt9916544            6.8        70   
620974  tt9916730            7.6        11   

                                          credited_actors  
0                                            [Carmencita]  
1       [Émile Reynaud, Gaston Paulin, Julien Pappé, T...  
2                                         [Gaston Paulin]  
3                              [Charles Kayser, John Ott]  
4                                        [Peter Courtney]  
...                                                   ...  
620970                    [

In [16]:
aggregated_df.head()

,tconst,averageRating,numVotes,credited_actors
0,tt0000001,5.7,2008,[Carmencita]
1,tt0000003,6.5,1925,"[Émile Reynaud, Gaston Paulin, Julien Pappé, T..."
2,tt0000004,5.4,178,[Gaston Paulin]
3,tt0000005,6.2,2698,"[Charles Kayser, John Ott]"
4,tt0000007,5.4,842,[Peter Courtney]


In [18]:
unique_tuples = set(tuple(x) for x in aggregated_df['credited_actors'])


unique_count = len(unique_tuples)

print(unique_count)


615884
